In [ ]:
# installing pytorch lightning and transformer
! pip install --quiet lightning
! pip install --quiet transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.6/801.6 kB 24.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
# importing all necessary packages
from os import listdir
from os.path import join
from sklearn.model_selection import train_test_split
import string
from torch.utils.data import Dataset, DataLoader
import torch
import torchmetrics
import pytorch_lightning as pl
from transformers import AutoModelForSequenceClassification
from pytorch_lightning.loggers import TensorBoardLogger, WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# For Google Colaboratory
import sys, os
if 'google.colab' in sys.modules:
    # mount google drive
    from google.colab import drive
    drive.mount('/content/gdrive')
    path_to_file = '/content/gdrive/My Drive/BT5151/Group project'
    print(path_to_file)
    # move to Google Drive directory
    os.chdir(path_to_file)
    !pwd

Mounted at /content/gdrive
/content/gdrive/My Drive/BT5151/Group project
/content/gdrive/My Drive/BT5151/Group project


In [ ]:
%run tokenizer_class.py

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

In [ ]:
from tokenizer_class import tokenize_text_with_labels

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding, Layer, Dense, Dropout, MultiHeadAttention, LayerNormalization, Input, GlobalAveragePooling1D
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizerFast, DistilBertModel

In [ ]:
reviews_df = pd.read_csv('reviews_df.csv')

In [ ]:
reviews_df.head()

,ProductId,UserId,Score,Time_ymd,Summary_Text,Clean_Text,Original_Text_Length,Clean_Text_Length,Helpfulness_Ratio,Sentiment,NumUniqueProducts,Days_since_last_review,polarity,subjectivity
0,B001E4KFG0,A3SGXH7AUHU8GW,5,2011-04-27,Good Quality Dog Food I have bought several of...,good quality food bought several vitality cann...,285,162,1.0,1,1,248,0.480000,0.440000
1,B000UA0QIQ,A395BORC6FGVXV,2,2011-06-13,Cough Medicine If you are looking for the secr...,cough medicine looking secret ingredient robit...,234,139,1.0,-1,3,201,0.150000,0.650000
2,B000E7L2R4,A1MZYO9TZK0BBI,5,2011-11-23,Yay Barley Right now I'm mostly just sprouting...,barley right mostly sprouting cats grass love ...,142,70,1.0,1,2,37,0.428571,0.545238
3,B001GVISJM,A2A9X58G2GTBLP,5,2011-12-23,"GREAT SWEET CANDY! Twizzlers, Strawberry my ch...",great sweet candy twizzlers strawberry childho...,765,492,-1.0,1,44,8,0.271429,0.550000
4,B001GVISJM,A3IV7CL2C13K2U,5,2011-10-08,Home delivered twizlers Candy was delivered ve...,home delivered twizlers candy delivered fast p...,169,103,-1.0,1,1,84,0.225000,0.675000


In [ ]:
reviews_df.shape

(161599, 14)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(reviews_df['Summary_Text'], reviews_df['Sentiment'], test_size=0.2,shuffle=True,random_state=42)
train_df = pd.DataFrame({'Text': X_train, 'Sentiment': y_train})
test_df = pd.DataFrame({'Text': X_test, 'Sentiment': y_test})

In [ ]:
train_df

,Text,Sentiment
14579,Tasty but it does contain MSG This bouillon is...,1
18331,"Hey, you got your fish in my peanut butter! So...",0
78945,Love it! I love this brand. I buy several diff...,1
123048,Guilt-Free but not that great I loved the idea...,0
74547,delicious This is a GREAT & delicious snack. ...,1
...,...,...
119879,Delicious high-quality chococate with a fascin...,1
103694,"Not bad...not for everyday, for me :-) It was ...",0
131932,terrific variety for coffee lovers There is so...,1
146867,"Excellent Service, Excellent Vendor! This Vend...",1


In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [ ]:
model_class, tokenizer_class = DistilBertModel, DistilBertTokenizerFast
tokenizer = tokenizer_class.from_pretrained("distilbert-base-uncased")
model = model_class.from_pretrained("distilbert-base-uncased", num_labels=3)
model = model.to(device)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
test = train_df.Text[2]
tok_text = tokenizer.encode_plus(
    test,
    add_special_tokens=True,
    padding="max_length",
    truncation=True,
    return_attention_mask=True,
    return_tensors="pt",
).to(device)

In [ ]:
with torch.no_grad():
    last_hidden_states = model(**tok_text)

In [ ]:
features = last_hidden_states[0][:, 0, :].cpu().numpy()
features.shape

(1, 768)

# Extract Features and Train Simple Classifier

In [ ]:
BATCH_SIZE = 100
train_chunks = [
    X_train[i : i + BATCH_SIZE]
    for i in range(0, int(X_train.shape[0]), BATCH_SIZE)
]
test_chunks = [
    X_test[i : i + BATCH_SIZE] for i in range(0, int(X_test.shape[0]), BATCH_SIZE)
]

In [ ]:
y_train_chunks = [
    y_train[i : i + BATCH_SIZE]
    for i in range(0, int(y_train.shape[0]), BATCH_SIZE)
]
y_test_chunks = [
    y_test[i : i + BATCH_SIZE] for i in range(0, int(y_test.shape[0]), BATCH_SIZE)
]

y_train_chunks = np.concatenate(y_train_chunks)
y_test_chunks = np.concatenate(y_test_chunks)

In [ ]:
import tqdm

# Extract train data features
train_features = []
for chunk in tqdm.tqdm(train_chunks):
    tok_text = tokenizer.batch_encode_plus(
        chunk.tolist(),
        add_special_tokens=True,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        return_tensors="pt",
    )

    tok_text = tok_text.to(device)
    with torch.no_grad():
        last_hidden_states = model(**tok_text)

    features = last_hidden_states[0][:, 0, :].cpu().numpy()
    train_features.append(features)


train_features = np.concatenate(train_features)
train_features.shape

100%|██████████| 1293/1293 [10:36<00:00,  2.03it/s]


(129279, 768)

In [ ]:
test_features = []
for chunk in tqdm.tqdm(test_chunks):
    tok_text = tokenizer.batch_encode_plus(
        chunk.tolist(),
        add_special_tokens=True,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        return_tensors="pt",
    )

    tok_text = tok_text.to(device)
    with torch.no_grad():
        last_hidden_states = model(**tok_text)

    features = last_hidden_states[0][:, 0, :].cpu().numpy()
    test_features.append(features)


test_features = np.concatenate(test_features)
test_features.shape

100%|██████████| 324/324 [02:39<00:00,  2.04it/s]


(32320, 768)

In [ ]:
# Save them in numpy data format

np.save("train_features.npy", train_features)
np.save("test_features.npy", test_features)

# Load embeddings

In [ ]:
# load train and validation features
train_features = np.load("train_features.npy")
test_features = np.load("test_features.npy")

# Logistic Regression

In [ ]:
# Evaluate the model based on accuray, precision, recall, F1 and ROC score
def evaluate_model(x_test, y_true, model,modelname):
  from sklearn.metrics import confusion_matrix, precision_score, accuracy_score, recall_score, f1_score, roc_auc_score ,classification_report, RocCurveDisplay
  y_pred = model.predict(x_test)
  print(modelname)
  precision = precision_score(y_true, y_pred,average='weighted')
  print('Precision: {:.2f}%'.format(precision))
  recall = recall_score(y_true, y_pred, average='weighted')
  print('Recall: {:.2f}%'.format( recall))
  f1 = f1_score(y_true, y_pred,average='weighted')
  print('F1_score: {:.2f}%'.format(f1))


  print('**********************************************************')
  print('*****************  Classification Report  ****************')
  print('**********************************************************')
  print(classification_report(y_true,y_pred))

In [ ]:
## Train a model on the training features.

train_features = train_features.reshape(train_features.shape[0], -1)
test_features = test_features.reshape(test_features.shape[0], -1)

modelname = "Logistic Regression"
log = LogisticRegression(penalty ='elasticnet',solver = 'saga', C= 0.1, l1_ratio = 0.5)
log.fit(train_features, y_train_chunks)
evaluate_model(test_features, y_test_chunks,log, modelname )

Logistic Regression
Precision: 0.85%
Recall: 0.87%
F1_score: 0.86%
**********************************************************
*****************  Classification Report  ****************
**********************************************************
              precision    recall  f1-score   support

          -1       0.75      0.73      0.74      5015
           0       0.53      0.23      0.32      2503
           1       0.91      0.97      0.94     24802

    accuracy                           0.87     32320
   macro avg       0.73      0.64      0.66     32320
weighted avg       0.85      0.87      0.86     32320



In [ ]:
train_features = train_features.reshape(train_features.shape[0], -1)
test_features = test_features.reshape(test_features.shape[0], -1)

modelname = "Logistic Regression"
log = LogisticRegression()
log.fit(train_features, y_train_chunks)
evaluate_model(test_features, y_test_chunks,log, modelname )

Logistic Regression
Precision: 0.85%
Recall: 0.87%
F1_score: 0.85%
**********************************************************
*****************  Classification Report  ****************
**********************************************************
              precision    recall  f1-score   support

          -1       0.74      0.72      0.73      5015
           0       0.49      0.23      0.32      2503
           1       0.91      0.96      0.93     24802

    accuracy                           0.87     32320
   macro avg       0.71      0.64      0.66     32320
weighted avg       0.85      0.87      0.85     32320

